### 載入cifar10

In [148]:
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf

(X_train, y_train),(X_test, y_test) = cifar10.load_data()

### 預處理

In [149]:
X_train = X_train.astype('float32')/255.0
X_test = X_test.astype('float32')/255.0
print("X_train_shape:{},X_test_shape:{}".format(X_train.shape,X_test.shape))

import keras
y_train = keras.utils.to_categorical(y_train,num_classes = 10)
y_test = keras.utils.to_categorical(y_test,num_classes = 10)

X_train_shape:(50000, 32, 32, 3),X_test_shape:(10000, 32, 32, 3)


### 定義模型變數

In [150]:
X = tf.placeholder(tf.float32, [None,32,32,3])
y_true = tf.placeholder(tf.float32, [None,10])

global_step = tf.Variable(initial_value=0, trainable=False, name='global_step')
EPOCHS = 4000
BATCH_SIZE = 128
CURR_POS = 0
TOTAL_NUMS = X_train.shape[0]
STEPS = TOTAL_NUMS//BATCH_SIZE

### 定義模型函式

In [151]:
def conv2d(x,w):
    return tf.nn.conv2d(x,w,strides = [1,2,2,1],padding = 'SAME')

def maxpool2d(x):
    return tf.nn.max_pool(x,ksize = [1,2,2,1],strides = [1,2,2,1],padding = 'SAME')

def weight_variable(shape, name):
    return tf.Variable(tf.truncated_normal(shape = shape,stddev = 0.1),name = name)

def bias_variable(shape, name):
    return tf.Variable(tf.constant(0.1,shape = shape),name = name)

def get_batch():
    global BATCH_SIZE,CURR_POS
    
    if CURR_POS+BATCH_SIZE<TOTAL_NUMS:
        batchx,batchy = X_train[CURR_POS:CURR_POS+BATCH_SIZE],y_train[CURR_POS:CURR_POS+BATCH_SIZE]
        CURR_POS+=BATCH_SIZE
    else:
        batchx,batchy = X_train[CURR_POS:],y_train[CURR_POS:]
        CURR_POS = 0

    return batchx,batchy

### 定義模型架構

#### 第一層捲基

In [152]:
w1 = weight_variable([5,5,3,32],'w1')
b1 = bias_variable([32],'b1')
conv1 = tf.nn.relu(conv2d(X,w1)+b1)
conv1_with_dropout = tf.nn.dropout(conv1,keep_prob)
o1 = maxpool2d(conv1_with_dropout)

#### 第二層捲基

In [153]:
w2 = weight_variable([5,5,32,64],'w2')
b2 = bias_variable([64],'b2')
conv2 = tf.nn.relu(conv2d(o1,w2)+b2)
conv2_with_dropout = tf.nn.dropout(conv2,keep_prob)
o2 = maxpool2d(conv2_with_dropout)

#### 撫平層

In [154]:
fc_in = tf.reshape(o2,[-1,2*2*64])
fc_w1 = weight_variable([2*2*64,1024], name = 'fc_w1')
fc_b1 = bias_variable([1024], name = 'fc_b1')
fc_out1 = tf.nn.relu(tf.matmul(fc_in,fc_w1)+fc_b1)
fc_out1_with_dropout = tf.nn.dropout(fc_out1,keep_prob)

#### 輸出層

In [155]:
fc_w2 = weight_variable([1024,10],name = 'fc_w2')
fc_b2 = bias_variable([10],name = 'fc_b2')
y_predict = tf.nn.softmax(tf.matmul(fc_out1_with_dropout,fc_w2)+fc_b2)

### 損失函數與優化器

In [156]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = y_predict, labels = y_true))
opt = tf.train.AdamOptimizer(0.0001).minimize(loss)

correct_prediction = tf.equal(tf.argmax(y_predict,1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### 實例化與執行

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(EPOCHS):
        for step in range(STEPS):
            batchx,batchy = get_batch()
            sess.run(opt,feed_dict = {X:batchx, y_true:batchy,keep_prob:0.5})
            
            if step%1000 == 0:
                print("Step:{},Accuracy:{}".format(step,sess.run(accuracy,feed_dict = {X:batchx,y_true:batchy,keep_prob:1.0})))
        
    print("Test accuracy:{}".format(sess.run(loss,feed_dict = {X:X_test,y_true:y_test,keep_prob:1.0})))

Step:0,Accuracy:0.078125
